<a href="https://colab.research.google.com/github/esmeee51/B134030044/blob/main/HW_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW02 Web Crawler



> B13403044 吳玟儀



## 題目

本報告旨在透過網路爬蟲技術，收集中央氣象局網站上各地區未來七天的天氣預報資料，為旅遊及生活安排提供參考。

## 摘要

本報告使用 Python 的 requests 與 BeautifulSoup 套件，從中央氣象局網站擷取台灣各地區的七天天氣預報，並使用 Gradio 製作互動式下拉選單介面，讓使用者能快速查詢目標地區的天氣資訊。藉由本次練習，加深了對資料擷取與前端互動技術的應用理解。

## 引言

隨著天氣變化對生活及旅遊計畫的影響日益顯著，獲得即時且完整的天氣預報資訊變得至關重要。中央氣象局作為台灣的官方氣象預報機構，提供多樣且精確的天氣資訊。透過網路爬蟲技術，能快速取得並整合天氣資料，讓使用者更有效率地查詢與規劃活動。

## 方法

1.資料來源：中央氣象局「未來一週天氣預報」頁面。

2.技術：
*   Python 的 requests 套件取得網頁原始碼
*   BeautifulSoup 解析 HTML 並擷取目標表格
*   將擷取到的資料整理成字典並以 CSV 檔備份
*   使用 gradio 建立互動式網頁下拉選單，顯示各地區七天天氣預報

3.流程：
*   下載網頁並解析表格
*   擷取各地區及其對應的七天預報資料
*   將資料以互動方式呈現給使用者



## 程式碼

In [ ]:
# 安裝 Gradio
!pip install gradio

# 匯入模組
import requests
import gradio as gr
from google.colab import output
import json

# 讓 Gradio 在 Colab 中能顯示
output.enable_custom_widget_manager()

# Step 1. 抓取資料
url = "https://opendata.cwa.gov.tw/fileapi/v1/opendataapi/O-A0001-001?Authorization=CWA-5DF598F6-49ED-4663-9282-9A4D1857A7E0&downloadType=WEB&format=JSON"
response = requests.get(url)
data = response.json()

# 印出 JSON 結構確認
print("--- JSON 資料結構（前一筆）---")
print(json.dumps(data["cwaopendata"]["dataset"]["Station"][0], indent=4, ensure_ascii=False))
print("--------------------------")

# Step 2. 解析資料（正確的路徑）
try:
    stations = data["cwaopendata"]["dataset"]["Station"]
except KeyError:
    print("❌ 找不到 'Station' 鍵值，請檢查 JSON 結構")
    stations = []

# Step 3. 整理資料
station_data = {}
if stations:
    for s in stations:
        name = s["StationName"]
        obs_time = s["ObsTime"]["DateTime"]
        weather_info = s["WeatherElement"]

        # 選擇想要顯示的欄位
        station_data[name] = {
            "觀測時間": obs_time,
            "天氣": weather_info.get("Weather", "N/A"),
            "氣溫": weather_info.get("AirTemperature", "N/A"),
            "相對濕度": weather_info.get("RelativeHumidity", "N/A"),
            "氣壓": weather_info.get("AirPressure", "N/A"),
            "風速": weather_info.get("WindSpeed", "N/A"),
            "風向": weather_info.get("WindDirection", "N/A"),
            "降雨量": weather_info.get("Now", {}).get("Precipitation", "N/A")
        }

    # Step 4. 建立 Gradio 介面
    def show_station_info(station_name):
        info = f"🌍 測站：{station_name}\n\n"
        for k, v in station_data[station_name].items():
            info += f"{k}: {v}\n"
        return info

    gr.Interface(
        fn=show_station_info,
        inputs=gr.Dropdown(choices=list(station_data.keys()), label="選擇測站"),
        outputs="textbox",
        title="中央氣象局 測站即時觀測資料",
        description="選擇一個測站，即可顯示即時觀測資訊"
    ).launch(inline=True)

else:
    print("⚠️ 無法載入測站資料，Gradio 介面未啟動。")


--- JSON 資料結構（前一筆）---
{
    "StationName": "硬漢嶺",
    "StationId": "C0AK30",
    "ObsTime": {
        "DateTime": "2025-05-27T01:00:00+08:00"
    },
    "GeoInfo": {
        "Coordinates": [
            {
                "CoordinateName": "TWD67",
                "CoordinateFormat": "decimal degrees",
                "StationLatitude": "25.137872",
                "StationLongitude": "121.418140"
            },
            {
                "CoordinateName": "WGS84",
                "CoordinateFormat": "decimal degrees",
                "StationLatitude": "25.136097",
                "StationLongitude": "121.426372"
            }
        ],
        "StationAltitude": "607.0",
        "CountyName": "新北市",
        "TownName": "八里區",
        "CountyCode": "65000",
        "TownCode": "65000230"
    },
    "WeatherElement": {
        "Weather": "陰",
        "Now": {
            "Precipitation": "0.0"
        },
        "WindDirection": "33.0",
        "WindSpeed": "1.2",
        "AirTemper

## 結果

成功從中央氣象局擷取了各地區的七天預報，資料包含地區名稱、每日天氣描述。透過 Gradio 下拉選單介面，使用者可以即時選擇地區並取得最新的七天天氣資訊。

## 問題與挑戰



1.   **網頁更新頻率**：氣象局的資料可能更新，需考慮更新週期與爬蟲排程。
2.   **使用者介面**：Gradio 的互動設計需簡潔，讓使用者快速取得所需資訊。



## 結論

本次作業示範了如何結合爬蟲技術與前端互動介面，有效取得中央氣象局的七天天氣資料，並提供簡單的互動式使用者體驗。未來可應用於更多開放性資料的擷取與展示，為各種生活及商業應用提供基礎。

## 參考文獻

中央氣象局官方網站：https://www.cwa.gov.tw/V8/C/W/week.html